# Web-scraping: сбор данных из баз данных и интернет-источников

*Алла Тамбовцева, НИУ ВШЭ*

## Практикум 8. Работа с API ВКонтакте: собираем посты со стены, комментарии к постам и информацию о пользователях

## Подготовка к работе

Загружаем модули и библиотеки, необходимые для работы:

In [ ]:
import requests
import time
import pandas as pd

Для начала давайте посмотрим на документацию к API и посмотрим, как к нему формировать запросы: https://dev.vk.com/api/api-requests.

По [инструкции](https://github.com/allatambov/WebScrape25/blob/main/vk-tokens-new.pdf) мы можем получить доступ к API, создать приложение и забрать сервисный ключ *(service token)*. Если приложение уже создано, его можно найти в перечне, доступном по этой [ссылке](https://dev.vk.com/ru/admin/apps-list).

In [ ]:
# вставить сервисный ключ (service token)
serv_token = ""

Теперь токен доступа у нас есть, всё готово к работе!

## Часть 1: выгружаем посты со стены сообщества

На этом практическом занятии мы будем выгружать посты из сообщества [скалодрома Rock Zona](https://vk.com/rzclimbing). Сохраним в переменные версию API, ссылку для метода работы со стеной сообщества и название сообщества:

In [ ]:
v = "5.199"
main_wall = "https://api.vk.com/method/wall.get"
domain = "rzclimbing"

Знакомая нам функция `get()` из библиотеки `requests` умеет подставлять в запрос необходимые параметры и объединять их с помощью `?` и `&`. Сохраним необходимые параметры в виде словаря:

In [ ]:
params_wall = {"access_token" : serv_token, 
              "domain" : domain, 
              "count" : 100,
              "v" : v}

А теперь сформируем запрос и выгрузим результаты в формате JSON – в Python данные в таком формате будут представлены в виде словаря:

In [ ]:
req_wall = requests.get(main_wall, params = params_wall)

In [ ]:
json_wall = req_wall.json()
#json_wall

Извлечём из этого большого словаря элемент, который отвечает за общее число постов на стене:

In [ ]:
nposts = json_wall['response']['count']
print(nposts)

Теперь извлечём элемент, который хранит результаты – список из маленьких словарей с информацией о постах (1 словарь = 1 пост):

In [ ]:
items_wall = json_wall['response']['items']

Посмотрим на один элемент такого списка:

In [ ]:
i = items_wall[2]
i

### Задача 1

Извлеките из элемента `i` следующие компоненты:

* id поста (`id_`);
* дата поста (`date`);
* текст поста (`text`);
* число лайков (`nlikes`);
* число просмотров (`nviews`);
* число комментариев (`ncomments`).

In [ ]:
### YOUR CODE HERE ###

### Задача 2

Изучить один пост и понять, что нам от него нужно, это хорошо, но, конечно, мы захотим выгрузить все посты сразу, а уже потом разобраться, какую информацию о них нам оставить. Ограничения данного API таковы, что за один раз мы можем выгрузить только 100 постов. Хорошие новости: каждый раз при выгрузке мы можем начинать с того поста, на котором закончили, то есть сначала выгрузить первые 100 постов, потом – следующие 100 постов, и так до тех пор, пока не заполучим все. 

Общее число постов сохранено в `nposts`. Посчитайте, сколько раз нужно будет выполнить выгрузку по 100 постов, чтобы собрать все тексты, и сохраните его в переменную `iterate`.

In [ ]:
### YOUR CODE HERE ###

### Задача 3

Прочитайте в документации к API ВКонтакте про аргумент `offset` в методе `wall.get`. Используя полученную информацию и блоки кода ниже, выгрузите и сохраните в список `items_all` данные по всем постам на стене сообщества.

**Подсказка:** чтобы расширять список правильным образом, используйте метод `.extend()`, а не `.append()`, он добавляет не один элемент, а сразу несколько.

In [ ]:
params_wall_long = {"access_token" : serv_token, 
              "domain" : domain, 
              "count" : 100,
              "v" : v,
              "offset" : 0}

In [ ]:
items_all = []

for i in range(iterate):
    req_wall_long = requests.get(main_wall, params = params_wall_long)
    json_wall_long = req_wall_long.json()
    items_wall_long = json_wall_long['response']['items']
    
    ### YOUR CODE HERE ###
    
    time.sleep(1.5)
    print(i)

Проверяем длину списка – все ли посты собраны:

In [ ]:
len(items_all)

Не отбираем ничего на этом этапе, просто превращаем список словарей в датафрейм:

In [ ]:
df = pd.DataFrame(items_all)
df.head(10)

### Задача 4

Создайте на основе датафрейма `df` новый датафрейм `small` со следующими столбцами:

* id поста (`id`);
* дата поста (`date`);
* текст поста (`text`);
* число лайков (`nlikes`);
* число просмотров (`nviews`);
* число комментариев (`ncomments`).

**Подсказка:** отберите сначала все столбцы с нужной информацией, напишите функцию для извлечения только числа лайков/просмотров/комментариев и примените её, удалите лишние столбцы.

In [ ]:
### YOUR CODE HERE ###

### Задача 5

Выберите только те строки в полученном датафрейме, которые соответствуют постам с числом комментариев больше 0, и сохраните их в датафрейм `with_comm`.

In [ ]:
### YOUR CODE HERE ###

## Часть 2: собираем комментарии к постам

В целях экономии времени не будем брать весь датафрейм `with_comm`, для примера отберем первые 10 строк:

In [ ]:
with_comm10 = with_comm.head(10)

Заберем id постов и преобразуем их в список:

In [ ]:
ids = list(with_comm10["id"])

Возьмем один id для примера и сформулируем запрос для получения комментариев к посту с этим id:

In [ ]:
i = 4799

# метод wall.getComments
# owner_id: id сообщества
# post_id: id поста
# v: версия API, все та же
# access_token: токен доступа, все тот же
# count: число комментариев, берем 100, max возможное за раз
# thread_items_count: число ответов на комментарий, 10, max за раз

main_comm = "https://api.vk.com/method/wall.getComments"
params_comm = {"owner_id" : -38936316,
              "post_id" : i,
              "v" : v,
              "access_token" : serv_token,
              "count" : 100, 
              "thread_items_count" : 10}

Формулируем запрос, отправляем и смотрим на результат в виде словаря:

In [ ]:
req = requests.get(main_comm, params = params_comm)
req.json()

### Задача 6

Давайте считать, что 100 самых новых комментариев и 10 ответов на них нам достаточно (если нет – аргумент `offset` есть, задача аналогична предыдущей по сбору постов). Повторите эту операцию выше для всех id в списке `ids` и сформируйте список с результатами. Преобразуйте результат в датафрейм `df_comm`. 

In [ ]:
# по аналогии с циклом ранее, только ничего не увеличиваем
# подставляем на место post_id значение i

comments_all = []

for i in ids:
    params_comm = {"owner_id" : -38936316,
              "post_id" : i,
              "v" : v,
              "access_token" : serv_token,
              "count" : 100, 
              "thread_items_count" : 10}
    req = requests.get(main_comm, params = params_comm)
    
    ### YOUR CODE HERE ###
    
    time.sleep(1)

In [ ]:
df_comm = pd.DataFrame(comments_all)

### Задача 7

Объедините датафрейм с собранными комментариями `df_comm` и датафрейм `with_comm10` через функцию `merge()` из `pandas`.

In [ ]:
### YOUR CODE HERE ###

## Часть 3:  собираем информацию о пользователях

Для примера выберем пользователя с id 20473269 (пока не из датафрейма, это всего лишь я):

In [ ]:
j = 20473269

Сформируем запрос:

In [ ]:
# метод users.get
# все знакомые параметры
# в fields перечисляем дополнительню информацию,
# которую хотим получить
# перечисляем через запятую без пробелов

main_user = "https://api.vk.com/method/users.get"
params_user = {"access_token" : serv_token,
               "v" : v,
               "user_id" : j,
               "fields" : "bdate,city,home_town,universities"
}
req2 = requests.get(main_user, params = params_user)
req2.json()

### Задача 8

Выберите какого-нибудь пользователя, оставившего комментарий, из датафрейма и запросите по нему интересующую информацию.

In [ ]:
### YOUR CODE HERE ###